# Aggregating American Powerlifting Data From 2015 to Now
## A Python script to extract data from Unequipped American Powerlifters over the past five years

Author: Brian McAbee

Date: 10.24.2020

## Introduction

**TL;DR**: View and Download the dataset for unequipped American powerlifters [here](https://www.kaggle.com/brianmcabee/us-powerlifting-competition-data-2015now).


This notebook was created to aggregate specific data provided by the [OpenPowerlifting project](https://www.openpowerlifting.org) into its own dataset. You may download an entire copy of the data [here](https://gitlab.com/openpowerlifting/opl-data).

This python script written in this notebook retrieves relevant, accurate data from American Full-Power (Squat/Bench/Deadlift) competitors over the past few years. There have been no value modifications to the filtered data, as the sole purpose of this notebook is to only extract the relevant records that will be useful to myself, and hopefully others, for future analyses. The end result of this notebook is a new csv file with the relevant American powerlifting data. 

For the sake of data accuracy, the new csv file will contain information from 2015 to present. Keep in my this option can be modified with the change of a parameter or two (as well as other filters to be discussed throughout the notebook). The reason for creating a new file is because the orginial csv file is pretty big, sitting at a little over 390MB at the time this notebook was created. 

## Importing Pandas and Reading in the Correct File
This project utilizes pandas, one of the few core python modules that is used for data analysis. In the cell below, we will import pandas as well as read in the file containing the entire OpenPowerlifting dataset which was conveniently downloaded from the second hyperlink above. Optionally, quick summaries of the data can be viewed by uncommenting the last view lines of the cell (one at a time).

***Note:*** It seems as though current OpenPowerlifting file naming conventions for their datasets contain the date of the most recent data input, so comptetions after the date in the file may not be available.

In [2]:
# Import Pandas
import pandas as pd

# Read csv file into a DataFrame
pl_df = pd.read_csv('openpowerlifting-2020-10-16.zip',low_memory=True)

In [7]:
# Uncomment and run each line separately to view more information about entire dataframe
#pl_df.describe() # Show summary statistics of the data
#pl_df.head(5) # Show first 5 rows of dataset
#pl_df.shape
#pl_df.info()

# Getting the Right Data
Now that all powerlifting data has been read into a pandas dataframe, filtering what we need into a new dataframe can now occur. The *README.txt* file that accompanies the downloading of OpenPowerlifting's data provides a breakdown of each category and its respective entry. In total, there are 40 columns composed of  information such as basic lifter information, weights lifted, and competition location to select from.

The code below filters events based on the following criteria:
* Full power competitions
    * Event = SBD
* Competitors who compete unequipped and without wraps
    * Equipment = Raw
* The competition was held in the USA
    * MeetCountry = USA

These filters are all arbitrary, but I decided to proceed with aggregating this new dataset with them because of my future plans with the new data. Feel free to modify and adjust them as needed. 

After extracting all raw American SBD data, the next step is to drop columns that are deemed not needed for future analysis. These include Country, Glossbrenner, [Goodlift](https://www.powerlifting.sport/rules/codes/info/ipf-formula), and Meet Country. The country column was dropped because the new dataset already is composed of only competitors who've competed in the USA, so there is no need for repetition here. Glossbrenner is not a very used metric in the United States, so I elected to remove it as well. Removing the Goodlift category may be the most questionable out of the three, but for now I have decided to leave it out as is. In the future I may run this script again to re-include the column, but I do not forsee myself using it for analysis in the near future. A case could be made to keep all of these columns in the new dataset, which is very fair, but for the time being the aforementioned columns will still be removed from the dataset.

The last applied filter for the new dataset is removing powerlifting data pre-2015. The year 2015 is arbitrary, but the reason behind selecting this as the earliest year was because most the the data in this dataset, approximately 77% of it, is from the year 2015 onward.

After all the filters have been applied, functions will be called to drop duplicate records in the dataset and then sort by year. By default, pandas' sort_values() function sorts the column in ascending order. 

The final two print statements in the cell below are there to show you the file size, in megabytes, of the two dataframes used to. You will see how much bigger the original dataframe is compared to the new one.

In [4]:
# Make a new dataframe for raw, full power competitors in the USA only from year 2015 to 2020
sbd_df = pl_df.loc[(pl_df['Event'] == 'SBD') & 
                   (pl_df['Equipment'] == 'Raw') & 
                   (pl_df['MeetCountry'] == 'USA')]

# Drop unneeded columns
sbd_df = sbd_df.drop(['Country','Glossbrenner','Goodlift','MeetCountry'],axis=1)

# Convert Date column from string to datetime type
sbd_df['Date'] = pd.to_datetime(sbd_df['Date'])

# Remove data that is before 2015
sbd_df = sbd_df.loc[sbd_df['Date'].dt.year >= 2015]

# Drop duplicate values
sbd_df.drop_duplicates(inplace=True)

# run to verify duplicates are no longer there
# sum(sbd_df.duplicated())

# Sort Data by Date
sbd_df = sbd_df.sort_values('Date')

print('Full dataframe size: ' + str(int((pl_df.memory_usage().sum() / 1000000))) + 'MB')
print('American SBD dataframe size: ' + str(int((sbd_df.memory_usage().sum() / 1000000))) + 'MB')

Full dataframe size: 677MB
American SBD dataframe size: 53MB


## Saving Our New DataFrame to Its Own File
Run the cell below to export this new, aggregated dataset to its own csv file. 

In [9]:
#save sbd_df to csv
sbd_df.to_csv('usa_sbd_data_2020-10-16.csv')

# Wrapping up

And just like that, we have everything we need to analyze data from full-power powerlifting competitors in the USA over the past five years. This data should help mitigate the need for similar data extraction in future projects for individuals who want to generate analyses on American powerlifters and their performance. 

***NOTE:*** To update this script to include more accurate information, simply read in a new OpenPowerlifting csv file in the first cell and then rename your new csv file in the last code cell.

#### - Brian